In [36]:
from datasets import load_dataset
import datasets

### 1. FPB

In [37]:
dic = {
    0:"negative",
    1:'neutral',
    2:'positive',
}

In [38]:
fpb_datasets = load_dataset("financial_phrasebank", "sentences_50agree")
fpb_datasets = fpb_datasets["train"]
fpb_datasets = fpb_datasets.to_pandas()
fpb_datasets.columns = ["input", "output"]
fpb_datasets["output"] = fpb_datasets["output"].apply(lambda x:dic[x])
fpb_datasets["instruction"]  = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."
fpb_datasets = datasets.Dataset.from_pandas(fpb_datasets)
fpb_datasets = fpb_datasets.train_test_split(seed = 42)['train']
fpb_datasets

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 3634
})

In [39]:
train_dataset = datasets.concatenate_datasets([fpb_datasets]*6)   # we want each data source have similar number of samples
train_dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 21804
})

### 2. FiQA SA

In [40]:
def make_label(x):
    if x < - 0.1: return "negative"
    elif x >=-0.1 and x < 0.1: return "neutral"
    elif x >= 0.1: return "positive"

In [41]:
def add_instructions(x):
    if x == "post":
        return "What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}."
    else:
        return "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."

In [42]:
dataset = load_dataset('pauri32/fiqa-2018')
dataset = datasets.concatenate_datasets([dataset["train"], dataset["validation"] ,dataset["test"] ])
dataset = dataset.to_pandas()
dataset["output"] = dataset.sentiment_score.apply(make_label)
dataset["instruction"] = dataset.format.apply(add_instructions)
dataset = dataset[['sentence', 'output',"instruction"]]
dataset.columns = ["input", "output","instruction"]
dataset = datasets.Dataset.from_pandas(dataset)
dataset = dataset.train_test_split(0.226, seed = 42)['train']
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 938
})

In [43]:
tmp_dataset = datasets.concatenate_datasets([dataset]*21)
train_dataset = datasets.concatenate_datasets([train_dataset, tmp_dataset]) 
print(tmp_dataset.num_rows)
train_dataset

19698


Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 41502
})

### 3. TFNS

In [44]:
dic = {
    0:"negative",
    1:'positive',
    2:'neutral',
}

In [45]:
social_media_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')
social_media_dataset = social_media_dataset['train']
social_media_dataset = social_media_dataset.to_pandas()
social_media_dataset['label'] = social_media_dataset['label'].apply(lambda x:dic[x])
social_media_dataset['instruction'] = 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.'
social_media_dataset.columns = ['input', 'output', 'instruction']
social_media_dataset = datasets.Dataset.from_pandas(social_media_dataset)
social_media_dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9543
})

In [46]:
tmp_dataset = datasets.concatenate_datasets([social_media_dataset]*2)
train_dataset = datasets.concatenate_datasets([train_dataset,tmp_dataset]) 
print(tmp_dataset.num_rows)
train_dataset

19086


Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 60588
})

### 4. NWGI

In [47]:
finance_dataset = load_dataset('oliverwang15/news_with_gpt_instructions')
finance_dataset = finance_dataset['train'].to_pandas()
finance_dataset['output'] = finance_dataset['label']
finance_dataset["input"] = finance_dataset["news"]
finance_dataset["instruction"] = 'What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}, then provide some short reasons.'
finance_dataset = finance_dataset[['input', 'output', 'instruction']]
finance_dataset = datasets.Dataset.from_pandas(finance_dataset)
finance_dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 16184
})

In [48]:
train_dataset = datasets.concatenate_datasets([train_dataset, finance_dataset])
all_dataset = train_dataset.shuffle(seed = 42)
all_dataset.shape

(76772, 3)

In [49]:
# from huggingface_hub import notebook_login
# notebook_login()

In [50]:
# all_dataset.push_to_hub("fingpt_chatglm2_sentiment_instruction_lora_ft_dataset")

In [51]:
21804 + 19698 + 19086 + 16184

76772

### Make Dataset

In [52]:
import json
from tqdm.notebook import tqdm

In [53]:
def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [54]:
data_list = []
for item in all_dataset.to_pandas().itertuples():
    tmp = {}
    tmp["instruction"] = item.instruction
    tmp["input"] = item.input
    tmp["output"] = item.output
    data_list.append(tmp)

In [55]:
with open("dataset_new.jsonl", 'w') as f:
    for example in tqdm(data_list, desc="formatting.."):
        f.write(json.dumps(format_example(example)) + '\n')

formatting..:   0%|          | 0/76772 [00:00<?, ?it/s]

### Tokenize

In [56]:
import json
from tqdm.notebook import tqdm

import datasets
from transformers import AutoTokenizer, AutoConfig

# model_name = "THUDM/chatglm2-6b"
model_name = "EleutherAI/gpt-neox-20b"
jsonl_path = "dataset_new.jsonl"
save_path = 'fingpt_data'
max_seq_length = 2048
skip_overlength = True

In [57]:
def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

def read_jsonl(path, max_seq_length, skip_overlength=False):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

In [58]:
dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(jsonl_path, max_seq_length, skip_overlength)
    )
dataset.save_to_disk(save_path)

Generating train split: 0 examples [00:00, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/76772 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/76772 [00:00<?, ? examples/s]